# Tâche 7

Déterminer la bande interdite,

Trouver la direction dans laquelle la dernière (première) bande de valence (conduction) présente le plus et le moins de dispersion (insérer une figure indiquant la dispersion par des flèches),

Calculer la masse effective au sommet (base) de la dernière (première) bande de valence (conduction) en supposant une dispersion parabolique.

In [19]:
import numpy as np
from mp_api.client import MPRester
from lmapr1492 import *
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt 
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.electronic_structure.core import Spin
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.electronic_structure.plotter import BSPlotter


In [20]:
mp_key = "xaEW7gxeGjtHSTeJuWSz9Uf8JpzNlgtg"
mp_id = "mp-9382"

In [21]:
with MPRester(mp_key) as m:
    structure = m.get_structure_by_material_id(mp_id)
    el_bs = m.get_bandstructure_by_material_id(mp_id)
struct = SpacegroupAnalyzer(structure)
conv_structure = SpacegroupAnalyzer(structure).get_conventional_standard_structure()

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

# 1) Structure de bande électronique du  SrZrN2

In [ ]:
fig = BSPlotter(el_bs)
fig.get_plot()

<Axes: xlabel='$\\mathrm{Wave\\ Vector}$', ylabel='$\\mathrm{E\\ -\\ E_f\\ (eV)}$'>

# 2) Bande interdite

In [ ]:
print(el_bs.get_band_gap())

La bande interdite (ou gap de bande) correspond à l’intervalle énergétique séparant la bande de valence de la bande de conduction dans un matériau. Pour qu’un électron puisse passer de la bande de valence à la bande de conduction (et ainsi contribuer à la conduction électrique), il doit absorber une énergie au moins égale à celle de cette bande interdite.

Dans le cas étudié (SrZrN2), l’énergie de la bande interdite est Eg = 0.2325 eV. La transition électronique s’effectue entre le point Γ (sommet de la bande de valence) et le point F (base de la bande de conduction). Ce matériau est donc un semiconducteur, avec un gap inférieur à 1 eV.

La bande interdite est indirecte, le sommet de la bande de valence et la base de la bande de conduction ne se situent pas au même point dans l’espace réciproque. En d'autres termes, une bande interdite indirecte peut être décrite comme le phénomène dans lequel le maximum de la bande de valence ainsi que le minimum de la bande de conduction sont à différentes valeurs de k.

Dans les semi-conducteurs à bande interdite indirecte, la transition d'un électron de la bande de valence à la bande de conduction nécessite non seulement un apport d'énergie, mais aussi un changement de vecteur d'onde. Les photons, ayant un vecteur d'onde très faible, ne peuvent pas fournir ce changement. Par conséquent, un phonon doit intervenir pour compenser cette différence de vecteur d'onde

In [ ]:
print("VBM band index structure:", el_bs.get_vbm()['band_index'])
print("CBM band index structure:", el_bs.get_cbm()['band_index'])


In [ ]:
bs_plot_data = BSPlotter(el_bs).bs_plot_data()
distance = bs_plot_data.get('ticks').get('distance')
distances = bs_plot_data.get('distances')
energy = bs_plot_data.get('energy')


vbm_band_index = el_bs.get_vbm()['band_index'][Spin.up][-1]  # Dernière bande de valence
cbm_band_index = el_bs.get_cbm()['band_index'][Spin.up][0]   # Première bande de conduction


BSPlotter(el_bs).get_plot()


for i in range(len(distances)):
    plt.plot(distances[i], energy.get('1')[i][vbm_band_index], 'green') 
    plt.plot(distances[i], energy.get('1')[i][cbm_band_index], 'red') 


cbm_coord = bs_plot_data.get('cbm')
for (x, y) in cbm_coord:
    plt.plot(x, y, 'or', markersize=8)  


vbm_coord = bs_plot_data.get('vbm')
for (x, y) in vbm_coord:
    plt.plot(x, y, 'og', markersize=8)  


plt.title("Mise en évidence de la bande interdite indirecte dans la structure des bandes électroniques")
plt.fill_between(
    np.linspace(distance[0], distance[-1], 500), 
    0, 
    el_bs.get_band_gap().get('energy'), 
    color="orange"
)

plt.show()


bsplot = BSPlotter(el_bs)
bsdata = bsplot.bs_plot_data()

vbm = el_bs.get_vbm()
cbm = el_bs.get_cbm()


indices_vbm = list(vbm["band_index"].get(Spin.up, []))  
indices_cbm = list(cbm["band_index"].get(Spin.up, []))  


kpoint_vbm = vbm["kpoint"].cart_coords
kpoint_cbm = cbm["kpoint"].cart_coords


print("\nDernière bande de valence :")
print("Énergie du maximum de la bande de valence = {:.2f} eV".format(vbm["energy"]))
print("Vecteur k en coordonnées cartésiennes =", kpoint_vbm)
print("Indices des bandes de valence :", indices_vbm)

print("\nPremière bande de conduction :")
print("Énergie minimum de la bande de conduction = {:.2f} eV".format(cbm["energy"]))
print("Vecteur k en coordonnées cartésiennes =", kpoint_cbm)
print("Indices des bandes de conduction :", indices_cbm)


# 3) Dispersion

La dispersion dans les bandes de conduction et de valence décrit comment l’énergie des électrons varie en fonction de leur vecteur d’onde k. Cette relation, notée E(k), influence directement la mobilité des électrons et les propriétés de transport électrique d’un matériau. Un électron se déplace dans un matériau avec une vitesse de groupe proportionnelle à la pente de E(k). Plus cette pente est prononcée, plus l’électron est mobile. À l’inverse, une pente faible traduit une dispersion réduite et une mobilité plus limitée.

Dans un diagramme de bandes électroniques, une forte dispersion correspond à une bande inclinée, indiquant que les électrons peuvent se déplacer rapidement, une caractéristique favorable à la conductivité. En revanche, une bande plate reflète une faible dispersion, suggérant que les électrons sont plus localisés et moins aptes à se déplacer librement.

In [ ]:
x = np.unique(bsdata["ticks"]["distance"])  
y = bsdata["energy"]

num_points = len(x) - 1 

dx = np.zeros(num_points)
dy_vbm = np.zeros(num_points)
dy_cbm = np.zeros(num_points)
m_vbm = np.zeros(num_points)
m_cbm = np.zeros(num_points)

for i in range(num_points):
    dx[i] = x[i+1] - x[i]

for i in range(num_points):
    dy_vbm[i] = y["1"][i][15][-1] - y["1"][i][15][0] 
    dy_cbm[i] = y["1"][i][16][-1] - y["1"][i][16][0] 


for i in range(num_points):
    m_vbm[i] = dy_vbm[i] / dx[i]
    m_cbm[i] = dy_cbm[i] / dx[i]

idx_max_vbm = np.argmax(abs(m_vbm))
idx_min_vbm = np.argmin(abs(m_vbm))
idx_max_cbm = np.argmax(abs(m_cbm))
idx_min_cbm = np.argmin(abs(m_cbm))

print("En valeur absolue :")
print("La plus grande dispersion pour la bande de valence : {:.6f} (index {})".format(abs(m_vbm[idx_max_vbm]), idx_max_vbm))
print("La plus grande dispersion pour la bande de conduction : {:.6f} (index {})".format(abs(m_cbm[idx_max_cbm]), idx_max_cbm))
print("La plus faible dispersion pour la bande de valence : {:.6f} (index {})".format(abs(m_vbm[idx_min_vbm]), idx_min_vbm))
print("La plus faible dispersion pour la bande de conduction : {:.6f} (index {})".format(abs(m_cbm[idx_min_cbm]), idx_min_cbm))



In [ ]:
bsplot.get_plot()


plt.arrow(x[idx_max_vbm], y["1"][idx_max_vbm][15][0], dx[idx_max_vbm], dy_vbm[idx_max_vbm], 
          color='m', width=0.05, head_width=0.15, length_includes_head=True, label="VBM max")

plt.arrow(x[idx_min_vbm], y["1"][idx_min_vbm][15][0], dx[idx_min_vbm], dy_vbm[idx_min_vbm], 
          color='k', width=0.05, head_width=0.15, length_includes_head=True, label="VBM min") 

plt.arrow(x[idx_max_cbm], y["1"][idx_max_cbm][16][0], dx[idx_max_cbm], dy_cbm[idx_max_cbm], 
          color='b', width=0.05, head_width=0.15, length_includes_head=True, label="CBM max")

plt.arrow(x[idx_min_cbm], y["1"][idx_min_cbm][16][0], dx[idx_min_cbm], dy_cbm[idx_min_cbm], 
          color='y', width=0.05, head_width=0.15, length_includes_head=True, label="CBM min")

d = len(distances)

for i in range(d):
    plt.plot(distances[i], energy['1'][i][vbm_band_index], 'green') 
    plt.plot(distances[i], energy['1'][i][cbm_band_index], 'red')   


for (x, y) in bs_plot_data.get("vbm"):
    plt.plot(x, y, 'og', markersize=10)  

for (x, y) in bs_plot_data.get("cbm"):
    plt.plot(x, y, 'or', markersize=10) 

plt.title("Dispersions des bandes de valence et conduction")
plt.legend()
plt.show()


# 4) Masse effective

Il est souvent intéressant de décrire les électrons commes des particules libres dont la masse effective varie plutôt que des particules de masse fixée évoluant dans un potentiel. Cette masse effective peut être positive ou négative, supérieur ou inférieure à la masse réelle de l'électron.

La masse effective est définie par le tenseur d'ordre 2 des dérivées secondes de l'énergie E par rapport au vecteur d'onde k : 

$ m^*_{ij} = \left( \frac{\partial^2 E}{\partial k_i \partial k_j} \right)^{-1}$

Nous nous intéressons ici à la masse effective au sommet de la dernière bande de valence et au bas de la première bande de conduction. Afin de simplifier le calcul, nous supposons une dispersion parabolique, ce qui permet de modéliser la relation entre l’énergie et le vecteur d’onde sous la forme d’une parabole. Dans le cadre de cette hypothèse, la masse effective se calcule selon l'expression suivante : 

$ m^* = \frac{\hbar^2 \cdot k^2}{2 \cdot E(k)} $




In [ ]:
norm_k_vbm = np.linalg.norm(kpoint_vbm)
norm_k_cbm = np.linalg.norm(kpoint_cbm)

hbar = 6.5821e-16  
me = 9.11e-31  

m_vbm = (hbar**2 * norm_k_vbm**2) / (2 * abs(vbm["energy"]))
m_cbm = (hbar**2 * norm_k_cbm**2) / (2 * abs(cbm["energy"]))  


print("La masse effective au sommet de la dernière bande de valence est {:.2e} kg.".format(m_vbm))
print("La masse effective à la base de la première bande de conduction est {:.2e} kg.".format(m_cbm))
